In [2]:
import os

import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

import missingno as msno

from sklearn.model_selection import train_test_split, cross_val_score  
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score 

import numpy as np
import matplotlib.pyplot as plt

from pygam import * 

import catboost


In [3]:
os.chdir('/home/jlm217/rds/rds-mrc-bsu-csoP2nj6Y6Y/mimah/stopah/stopah/data') 

pd.set_option('display.max_columns', None)  

ro.r['load']('STOPAH_ForSolon.RData')

def R_dataset(x):
    with (ro.default_converter + pandas2ri.converter).context():
        stopah = ro.conversion.get_conversion().rpy2py(ro.r[x])
    return stopah

df = R_dataset('stopah')

R[write to console]: Warning:
R[write to console]:  namespace ‘pROC’ is not available and has been replaced
by .GlobalEnv when processing object ‘roc.objs’



In [4]:
sevenday = ['Hepatic.Encephalopathy...Treatment.Day.7..','Day.7.infection',
'Gastrointestinal.Bleed.since.the.last.visit.Gastrointestinal.bleed...and.Choose..Treatment.Day.7..',
'Temperature..Treatment.Day.7..','Pulse..Treatment.Day.7..',
'Systolic.BP..Treatment.Day.7..','Diastolic.BP..Treatment.Day.7..',
'MAP..Treatment.Day.7','Hb..1.decimal.point..Haematology..Treatment.Day.7..',
'Platelets.day.7','WBC.day.7','Neutrophils.day.7','INR.clinical.and.calc.day.7',
'Bilirubin.day.7','ALT.day.7','ALP.day.7','Albumin.day.7','Sodium.day.7','Potassium.day.7',
'Urea.day.7','Creatinine.day.7']

#reduce data set to target, treatment and therapy

df = df[sevenday+['D28_DTH','Prednisolone']]

df.reset_index(drop=True, inplace=True)

In [5]:
X, y = df.drop(['D28_DTH','Prednisolone'], axis=1), df[['D28_DTH']]

In [6]:
X[X['Bilirubin.day.7']==-2147483648]

,Hepatic.Encephalopathy...Treatment.Day.7..,Day.7.infection,Gastrointestinal.Bleed.since.the.last.visit.Gastrointestinal.bleed...and.Choose..Treatment.Day.7..,Temperature..Treatment.Day.7..,Pulse..Treatment.Day.7..,Systolic.BP..Treatment.Day.7..,Diastolic.BP..Treatment.Day.7..,MAP..Treatment.Day.7,Hb..1.decimal.point..Haematology..Treatment.Day.7..,Platelets.day.7,WBC.day.7,Neutrophils.day.7,INR.clinical.and.calc.day.7,Bilirubin.day.7,ALT.day.7,ALP.day.7,Albumin.day.7,Sodium.day.7,Potassium.day.7,Urea.day.7,Creatinine.day.7
4,-2147483648,0,-2147483648,NaN,-2147483648,-2147483648,-2147483648,NaN,-2147483648,-2147483648,NaN,NaN,NaN,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,NaN,NaN,-2147483648
11,0,0,-2147483648,36.8,89,127,69,88.333333,-2147483648,-2147483648,NaN,NaN,NaN,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,NaN,NaN,-2147483648
12,-2147483648,0,-2147483648,NaN,-2147483648,-2147483648,-2147483648,NaN,-2147483648,-2147483648,NaN,NaN,NaN,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,NaN,NaN,-2147483648
13,-2147483648,0,-2147483648,NaN,-2147483648,-2147483648,-2147483648,NaN,-2147483648,-2147483648,NaN,NaN,NaN,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,NaN,NaN,-2147483648
14,-2147483648,0,-2147483648,NaN,-2147483648,-2147483648,-2147483648,NaN,-2147483648,-2147483648,NaN,NaN,NaN,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,NaN,NaN,-2147483648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040,-2147483648,0,-2147483648,NaN,-2147483648,-2147483648,-2147483648,NaN,-2147483648,-2147483648,NaN,NaN,NaN,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,NaN,NaN,-2147483648
1043,-2147483648,0,-2147483648,NaN,-2147483648,-2147483648,-2147483648,NaN,-2147483648,-2147483648,NaN,NaN,NaN,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,NaN,NaN,-2147483648
1053,-2147483648,0,-2147483648,NaN,-2147483648,-2147483648,-2147483648,NaN,-2147483648,-2147483648,NaN,NaN,NaN,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,NaN,NaN,-2147483648
1059,-2147483648,0,-2147483648,NaN,-2147483648,-2147483648,-2147483648,NaN,-2147483648,-2147483648,NaN,NaN,NaN,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,NaN,NaN,-2147483648
